In [1]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps
from census import Census
from us import states

#import API Keys
from config import api_key
from config import gkey

#set census wrapper & gmaps settings
c = Census(api_key, year = 2016)
gmaps.configure(api_key = gkey)

#import commuter data csv
county_commute_data = pd.read_csv("..\Resources\county_commute_data.csv")

In [2]:
##plot heatmap of # of commuters
#set gmaps figure
fig = gmaps.figure()

#set coordinates as locations
locations = county_commute_data[["Latitude", "Longitude"]]

#plot heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=county_commute_data["Total Commuters"], 
                                 dissipating=False, max_intensity=max(county_commute_data["Total Commuters"]),
                                 point_radius = 1)

#adjust heat layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = max(county_commute_data["Total Commuters"])
heat_layer.point_radius = 1

#add heat layer to figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [3]:
##add metropolitan/non-metropolitan distinction
#create lists of metro counties
KC_metro_counties_MO = ["Bates County, Missouri", "Caldwell County, Missouri", 
                        "Cass County, Missouri", "Clay County, Missouri", 
                        "Clinton County, Missouri", "Jackson County, Missouri", 
                        "Lafayette County, Missouri", "Platte County, Missouri", 
                        "Ray County, Missouri"]
KC_metro_counties_KS = ["Johnson County, Kansas", "Leavenworth County, Kansas", 
                        "Linn County, Kansas", "Miami County, Kansas", 
                        "Wyandotte County, Kansas"]

STL_metro_counties_MO = ["St. Louis city, Missouri", "St. Louis County, Missouri", 
                         "St. Charles County, Missouri", "Jefferson County, Missouri", 
                         "Franklin County, Missouri", "Lincoln County, Missouri", 
                         "Warren County, Missouri"] 
STL_metro_counties_IL = ["Madison County, Illinois", "St. Clair County, Illinois", 
                         "Clinton County, Illinois", "Monroe County, Illinois", 
                         "Jersey County, Illinois"]

SPR_metro_counties = ["Greene County, Missouri", "Christian County, Missouri", 
                      "Webster County, Missouri", "Polk County, Missouri", 
                      "Dallas County, Missouri"]

#create table of all metro counties
metro_counties_MO = pd.DataFrame(KC_metro_counties_MO)
metro_counties_MO = metro_counties_MO.append(STL_metro_counties_MO)
metro_counties_MO = metro_counties_MO.append(SPR_metro_counties)

#set empty list for metro poles
metro_pole_list_MO = []

#fill metro pole list in order appended to table
for i in range(len(KC_metro_counties_MO)):
    metro_pole_list_MO.append("Kansas City")
    
for i in range(len(STL_metro_counties_MO)):
    metro_pole_list_MO.append("St. Louis")

for i in range(len(SPR_metro_counties)):
    metro_pole_list_MO.append("Springfield")
    
#rename column
metro_counties_MO.columns = ["Name"]
#add columns for metro pole and metro/rural
metro_counties_MO["Metro Pole"] = metro_pole_list_MO
metro_counties_MO["Metro/Rural"] = "Metro"

metro_counties_MO

,Name,Metro Pole,Metro/Rural
0,"Bates County, Missouri",Kansas City,Metro
1,"Caldwell County, Missouri",Kansas City,Metro
2,"Cass County, Missouri",Kansas City,Metro
3,"Clay County, Missouri",Kansas City,Metro
4,"Clinton County, Missouri",Kansas City,Metro
5,"Jackson County, Missouri",Kansas City,Metro
6,"Lafayette County, Missouri",Kansas City,Metro
7,"Platte County, Missouri",Kansas City,Metro
8,"Ray County, Missouri",Kansas City,Metro
0,"St. Louis city, Missouri",St. Louis,Metro


In [4]:
#merge MO metro data with county data
county_commute_data = county_commute_data.merge(metro_counties_MO, how = "outer")
#label non-metro counties rural
county_commute_data["Metro/Rural"].fillna("Rural", inplace=True)
#set column order
county_commute_data = county_commute_data[["Name", "Metro/Rural", "Metro Pole", "Total Population", 
                                           "Commuter %", "Total Commuters", 
                                           "Solo Commuter %", "Solo Commuters", 
                                           "Carpooler %", "Carpoolers", 
                                           "Public Transit %", "Public Transit",
                                           "Walking %", "Walking", "Other Transit %", 
                                           "Other Transit", "Latitude", "Longitude"]]
county_commute_data

#save to csv
county_commute_data.to_csv("..\Resources\county_commute_data.csv", index=False)

In [5]:
#locate only metro MO counties
metro_commute_data_MO = county_commute_data.loc[county_commute_data["Metro/Rural"] == "Metro"]
#save to csv
metro_commute_data_MO.to_csv("..\Resources\metro_commute_data_MO.csv", index=False)

In [6]:
#create table for non-MO metro counties
metro_counties_other = pd.DataFrame(KC_metro_counties_KS)
metro_counties_other = metro_counties_other.append(STL_metro_counties_IL)

#set empty list for metro poles
metro_pole_list_other = []

#fill metro pole list in order appended to table
for i in range(len(KC_metro_counties_KS)):
    metro_pole_list_other.append("Kansas City")
    
for i in range(len(STL_metro_counties_IL)):
    metro_pole_list_other.append("St. Louis")

#rename column
metro_counties_other.columns = ["Name"]
#add metro pole and metro/rural columns
metro_counties_other["Metro Pole"] = metro_pole_list_other
metro_counties_other["Metro/Rural"] = "Metro"

metro_counties_other

,Name,Metro Pole,Metro/Rural
0,"Johnson County, Kansas",Kansas City,Metro
1,"Leavenworth County, Kansas",Kansas City,Metro
2,"Linn County, Kansas",Kansas City,Metro
3,"Miami County, Kansas",Kansas City,Metro
4,"Wyandotte County, Kansas",Kansas City,Metro
0,"Madison County, Illinois",St. Louis,Metro
1,"St. Clair County, Illinois",St. Louis,Metro
2,"Clinton County, Illinois",St. Louis,Metro
3,"Monroe County, Illinois",St. Louis,Metro
4,"Jersey County, Illinois",St. Louis,Metro


In [7]:
#for FIP county codes: https://www.census.gov/geo/reference/codes/cou.html
county_codes_KS = ["091", "103", "107", "121", "209"]
county_codes_IL = ["119", "163", "027", "133", "083"]

#set empty data frame for non-MO county data
other_state_commute_data = pd.DataFrame()

#get non-MO county data
for i in range(len(county_codes_KS)):
    commute_type_data_KS = c.acs5.state_county(('NAME','B08301_001E',
                                             'B08301_003E', 'B08301_004E',
                                             'B08301_010E', 'B08301_019E',
                                             'B08101_041E', 'B01003_001E'), states.KS.fips, county_codes_KS[i])
    commute_type_data_IL = c.acs5.state_county(('NAME','B08301_001E',
                                             'B08301_003E', 'B08301_004E',
                                             'B08301_010E', 'B08301_019E',
                                             'B08101_041E', 'B01003_001E'), states.IL.fips, county_codes_IL[i])
    other_state_commute_data = other_state_commute_data.append(pd.DataFrame(commute_type_data_KS))
    other_state_commute_data = other_state_commute_data.append(pd.DataFrame(commute_type_data_IL))

#rename columns
other_state_commute_data = other_state_commute_data.rename(columns={"B08301_001E": "Total Commuters",
                                                                    "B08301_003E": "Solo Commuters",
                                                                    "B08301_004E": "Carpoolers",
                                                                    "B08301_010E": "Public Transit",
                                                                    "B08301_019E": "Walking",
                                                                    "B08101_041E": "Other Transit",
                                                                    "B01003_001E": "Total Population", 
                                                                    "NAME": "Name"})    
    
other_state_commute_data

,Total Population,Other Transit,Total Commuters,Solo Commuters,Carpoolers,Public Transit,Walking,Name,county,state
0,572428.0,2656.0,300970.0,256514.0,21077.0,1139.0,2588.0,"Johnson County, Kansas",091,20
0,266759.0,1191.0,123390.0,105337.0,8475.0,2080.0,1332.0,"Madison County, Illinois",119,17
0,78785.0,569.0,34739.0,28250.0,4177.0,60.0,700.0,"Leavenworth County, Kansas",103,20
0,265569.0,1619.0,117738.0,96571.0,8717.0,4990.0,2021.0,"St. Clair County, Illinois",163,17
0,9524.0,27.0,3961.0,3308.0,410.0,11.0,39.0,"Linn County, Kansas",107,20
0,37858.0,304.0,18495.0,15199.0,1801.0,115.0,403.0,"Clinton County, Illinois",027,17
0,32787.0,205.0,15867.0,13399.0,1248.0,22.0,179.0,"Miami County, Kansas",121,20
0,33703.0,52.0,17074.0,14454.0,1307.0,107.0,175.0,"Monroe County, Illinois",133,17
0,161777.0,495.0,71474.0,57876.0,9533.0,953.0,927.0,"Wyandotte County, Kansas",209,20
0,22441.0,60.0,9860.0,8342.0,700.0,18.0,335.0,"Jersey County, Illinois",083,17


In [8]:
#calculate commuter %
other_state_commute_data["Commuter %"] = (other_state_commute_data["Total Commuters"] 
                                          / other_state_commute_data["Total Population"] * 100)

#list commute type columns
commute_types = ["Solo Commuters", "Carpoolers", "Public Transit", "Walking", "Other Transit"]
#calculate commute type %s
commute_type_percents = [other_state_commute_data[commute_type] / other_state_commute_data["Total Commuters"] * 100 
                         for commute_type in commute_types]

#store %s in new columns per commute type
other_state_commute_data["Solo Commuter %"] = commute_type_percents[0]
other_state_commute_data["Carpooler %"] = commute_type_percents[1]
other_state_commute_data["Public Transit %"] = commute_type_percents[2]
other_state_commute_data["Walking %"] = commute_type_percents[3]
other_state_commute_data["Other Transit %"] = commute_type_percents[4]

#reset index
other_state_commute_data = other_state_commute_data.reset_index()
#set column order
other_state_commute_data = other_state_commute_data[["Name", "Total Population", 
                                                     "Commuter %", "Total Commuters", 
                                                     "Solo Commuter %", "Solo Commuters", 
                                                     "Carpooler %", "Carpoolers",  
                                                     "Public Transit %", "Public Transit",  
                                                     "Walking %", "Walking",
                                                     "Other Transit %", "Other Transit"]]
other_state_commute_data

,Name,Total Population,Commuter %,Total Commuters,Solo Commuter %,Solo Commuters,Carpooler %,Carpoolers,Public Transit %,Public Transit,Walking %,Walking,Other Transit %,Other Transit
0,"Johnson County, Kansas",572428.0,52.577791,300970.0,85.229093,256514.0,7.003024,21077.0,0.378443,1139.0,0.859886,2588.0,0.882480,2656.0
1,"Madison County, Illinois",266759.0,46.255234,123390.0,85.369155,105337.0,6.868466,8475.0,1.685712,2080.0,1.079504,1332.0,0.965232,1191.0
2,"Leavenworth County, Kansas",78785.0,44.093419,34739.0,81.320706,28250.0,12.023950,4177.0,0.172717,60.0,2.015026,700.0,1.637929,569.0
3,"St. Clair County, Illinois",265569.0,44.334241,117738.0,82.021947,96571.0,7.403727,8717.0,4.238224,4990.0,1.716523,2021.0,1.375087,1619.0
4,"Linn County, Kansas",9524.0,41.589668,3961.0,83.514264,3308.0,10.350921,410.0,0.277708,11.0,0.984600,39.0,0.681646,27.0
5,"Clinton County, Illinois",37858.0,48.853611,18495.0,82.178967,15199.0,9.737767,1801.0,0.621790,115.0,2.178967,403.0,1.643687,304.0
6,"Miami County, Kansas",32787.0,48.394181,15867.0,84.445705,13399.0,7.865381,1248.0,0.138653,22.0,1.128128,179.0,1.291990,205.0
7,"Monroe County, Illinois",33703.0,50.660179,17074.0,84.655031,14454.0,7.654914,1307.0,0.626684,107.0,1.024950,175.0,0.304557,52.0
8,"Wyandotte County, Kansas",161777.0,44.180570,71474.0,80.974900,57876.0,13.337717,9533.0,1.333352,953.0,1.296975,927.0,0.692560,495.0
9,"Jersey County, Illinois",22441.0,43.937436,9860.0,84.604462,8342.0,7.099391,700.0,0.182556,18.0,3.397566,335.0,0.608519,60.0


In [9]:
##find coordinates for each county
#set base url for google maps geocoding
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

#set empty lists for latitudes and longitudes
latitudes = []
longitudes = []

#list non-MO county names
counties = other_state_commute_data["Name"]

#query google maps geocoding with each county
for i in range(len(counties)):
    params = {"address": counties[i], "key": gkey}
    
    #get response and convert to json
    county_search = requests.get(base_url, params = params).json()
    
    #print request record
    print(f"Processing Record {i + 1}: {counties[i]}")
    
    #add data to latitudes and longitudes lists
    latitudes.append(county_search["results"][0]["geometry"]["location"]["lat"])
    longitudes.append(county_search["results"][0]["geometry"]["location"]["lng"])

Processing Record 1: Johnson County, Kansas
Processing Record 2: Madison County, Illinois
Processing Record 3: Leavenworth County, Kansas
Processing Record 4: St. Clair County, Illinois
Processing Record 5: Linn County, Kansas
Processing Record 6: Clinton County, Illinois
Processing Record 7: Miami County, Kansas
Processing Record 8: Monroe County, Illinois
Processing Record 9: Wyandotte County, Kansas
Processing Record 10: Jersey County, Illinois


In [10]:
#add coordinates to data frame
other_state_commute_data["Latitude"] = latitudes
other_state_commute_data["Longitude"] = longitudes

#merge non-MO metro counties tables
other_state_commute_data = other_state_commute_data.merge(metro_counties_other, how = "outer")
#set column order
other_state_commute_data = other_state_commute_data[["Name", "Metro/Rural", "Metro Pole", "Total Population", 
                                                     "Commuter %",  "Total Commuters", 
                                                     "Solo Commuter %","Solo Commuters", 
                                                     "Carpooler %", "Carpoolers",  
                                                     "Public Transit %", "Public Transit",  
                                                     "Walking %", "Walking","Other Transit %", 
                                                     "Other Transit", "Latitude", "Longitude"]]

#export csv
other_state_commute_data.to_csv("..\Resources\other_state_commute_data.csv", index=False)
other_state_commute_data

,Name,Metro/Rural,Metro Pole,Total Population,Commuter %,Total Commuters,Solo Commuter %,Solo Commuters,Carpooler %,Carpoolers,Public Transit %,Public Transit,Walking %,Walking,Other Transit %,Other Transit,Latitude,Longitude
0,"Johnson County, Kansas",Metro,Kansas City,572428.0,52.577791,300970.0,85.229093,256514.0,7.003024,21077.0,0.378443,1139.0,0.859886,2588.0,0.882480,2656.0,38.845412,-94.852064
1,"Madison County, Illinois",Metro,St. Louis,266759.0,46.255234,123390.0,85.369155,105337.0,6.868466,8475.0,1.685712,2080.0,1.079504,1332.0,0.965232,1191.0,38.904140,-89.925323
2,"Leavenworth County, Kansas",Metro,Kansas City,78785.0,44.093419,34739.0,81.320706,28250.0,12.023950,4177.0,0.172717,60.0,2.015026,700.0,1.637929,569.0,39.171307,-95.018246
3,"St. Clair County, Illinois",Metro,St. Louis,265569.0,44.334241,117738.0,82.021947,96571.0,7.403727,8717.0,4.238224,4990.0,1.716523,2021.0,1.375087,1619.0,38.400812,-89.925323
4,"Linn County, Kansas",Metro,Kansas City,9524.0,41.589668,3961.0,83.514264,3308.0,10.350921,410.0,0.277708,11.0,0.984600,39.0,0.681646,27.0,38.133957,-94.810595
5,"Clinton County, Illinois",Metro,St. Louis,37858.0,48.853611,18495.0,82.178967,15199.0,9.737767,1801.0,0.621790,115.0,2.178967,403.0,1.643687,304.0,38.568444,-89.625165
6,"Miami County, Kansas",Metro,Kansas City,32787.0,48.394181,15867.0,84.445705,13399.0,7.865381,1248.0,0.138653,22.0,1.128128,179.0,1.291990,205.0,38.638169,-94.810595
7,"Monroe County, Illinois",Metro,St. Louis,33703.0,50.660179,17074.0,84.655031,14454.0,7.654914,1307.0,0.626684,107.0,1.024950,175.0,0.304557,52.0,38.274253,-90.186964
8,"Wyandotte County, Kansas",Metro,Kansas City,161777.0,44.180570,71474.0,80.974900,57876.0,13.337717,9533.0,1.333352,953.0,1.296975,927.0,0.692560,495.0,39.098681,-94.769159
9,"Jersey County, Illinois",Metro,St. Louis,22441.0,43.937436,9860.0,84.604462,8342.0,7.099391,700.0,0.182556,18.0,3.397566,335.0,0.608519,60.0,39.070475,-90.374835
